## LLM Agents

* Local implementation following this tutorial: https://graphacademy.neo4j.com/courses/llm-fundamentals/3-intro-to-langchain/7-c-neo4j-vector-retriever/

### Requirements

In [1]:
!pip install langchain openai langchain-openai neo4j python-dotenv langchainhub langchain-community --quiet

In [1]:
%load_ext watermark
%watermark -p langchain,langchainhub,langchain_community

langchain          : 0.1.5
langchainhub       : 0.1.14
langchain_community: 0.0.17



### Imports

In [2]:
import os
from graphdatascience import GraphDataScience
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
import neo4j

### Settings

In [3]:
project_path = Path(os.getcwd()).parent
data_path = project_path / "data"
model_path = project_path / "models"
output_path = project_path / "output"

# load env settings
_ = load_dotenv(find_dotenv())

llm_model = "gpt-4"
database = "recommendations-50"

openai_api_key = os.getenv('OPENAI_API_KEY')

### Connect to Neo4j

In [4]:
from langchain_community.graphs import Neo4jGraph

#### Solution

In [5]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, RetrievalQA
#from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub
from langchain_community.tools import YouTubeSearchTool
from langchain_community.vectorstores.neo4j_vector import Neo4jVector

llm = ChatOpenAI(openai_api_key=openai_api_key)
embedding_provider = OpenAIEmbeddings(openai_api_key=openai_api_key)

prompt = PromptTemplate(
    template="""
    You are a movie expert. You find movies from a genre or plot. 

    Question:{input}
    """, 
    input_variables=["input"]
    )

#memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
#memory = CustomChatMemory(memory_key="chat_history", return_messages=True)

chat_chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

youtube = YouTubeSearchTool()

movie_plot_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    url=os.getenv('NEO4J_URL'),
    username=os.getenv('NEO4J_USER'),
    password=os.getenv('NEO4J_PASS'),
    index_name="moviePlots",
    embedding_node_property="embedding",
    text_node_property="plot",
)

plot_retriever = RetrievalQA.from_llm(
    llm=llm,
    retriever=movie_plot_vector.as_retriever(), 
    verbose=True, 
    return_source_documents=True
)

def run_retriever(query):
    results = plot_retriever.invoke({"query":query})
    # format the results
    movies = '\n'.join([doc.metadata["title"] + " - " + doc.page_content for doc in results["source_documents"]])
    return movies

tools = [
    # Tool.from_function(
    #     name="Movie Chat",
    #     description="For when you need to chat about movies. The question will be a string. Return a string.",
    #     func=chat_chain.run,
    #     return_direct=True
    # ),
    Tool.from_function(
        name="Movie Trailer Search",
        description="Use when needing to find a movie trailer. The question will include the word 'trailer'. Return a link to a YouTube video.",
        func=youtube.run,
        return_direct=True
    ),
    Tool.from_function(
        name="Movie Plot Search",
        description="For when you need to compare a plot to a movie. The question will be a string. Return a string.",
        func=run_retriever,
        return_direct=True
    )
]

agent_prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    #memory=memory,
    max_interations=3,
    verbose=True,
    handle_parse_errors=True,
    #return_intermediate_steps=True
)

q = "I am looking for a movie where a trip to the moon goes wrong"
response = agent_executor.invoke({"input": q})
print(response["output"])



> Entering new AgentExecutor chain...


KeyError: "Input to PromptTemplate is missing variables {'chat_history'}.  Expected: ['agent_scratchpad', 'chat_history', 'input'] Received: ['input', 'intermediate_steps', 'agent_scratchpad']"

In [ ]:
agent_executor.invoke({"input": q})

In [ ]:
print(response)